In [1]:
import requests
from requests.auth import HTTPProxyAuth
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
from random import randint
import pandas as pd
import re
import socks
import socket
import sqlite3
from stem import Signal
from stem.control import Controller 
import numpy as np

In [10]:
## Initizlization

#Create SQLite
conn = sqlite3.connect('lyrics.db')
cursor = conn.cursor()
# df = pd.read_sql_query("select * from lyrics_url;",conn)

In [23]:
db_name = "lyrics_abc"

In [ ]:
## Steps for Git
Git add <file> 
Git commit -m "comments" 
Git push origin master

## Scrap Cyber Hymnal

In [3]:
def set_new_ip():
    """Change IP using TOR"""
    with Controller.from_port(port=9051) as controller:
        controller.authenticate()
        controller.signal(Signal.NEWNYM)


In [4]:
#start tor :  tor & 
#netstat -tupln 
# http://sacharya.com/crawling-anonymously-with-tor-in-python/
# socks.setdefaultproxy(proxy_type=socks.PROXY_TYPE_SOCKS5, addr="127.0.0.1", port=9050)
# socket.socket = socks.socksocket
session = requests.session()
session.proxies = {'https':'socks5://127.0.0.1:9050','http':'socks5://127.0.0.1:9050'}


In [14]:
set_new_ip()
print(session.get("http://icanhazip.com").text)
set_new_ip()
print(session.get("http://icanhazip.com").text)

5.135.158.101

94.23.173.249



In [17]:
columns = ['url','title','status']
df = pd.DataFrame(columns=columns)

url = 'http://cyberhymnal.org/ttl/ttl-'
for i in range(ord('y'), ord('z')+1):
    #    print(url+chr(i)+'.htm')
    try:
        hurl = url+chr(i)+'.htm'
        pause = randint(3,8)
        time.sleep(pause)
        response = session.get(hurl, headers=headers)
        data = response.text
        soup = BeautifulSoup(data,"lxml")
        div =  soup.find('div',attrs={"class":"single-entry"})
        links = div.findAll("a")
        #Loop through the link in each page
        for i in links:
            url_link =  i.get("href").replace("..","http://cyberhymnal.org")
            song_name = i.get_text()
            status = 0
            row = [url_link,song_name,status]
            # print(row)
            df.loc[len(df)] = row  
    except Exception as e:
        print(hurl)
        print(e)

len(df)
# df.reset_index(inplace=True)
# df['key'] = df['index']
# del df['index']

# df.to_sql('lyrics_url',conn,if_exists='replace')
# df.to_sql('lyrics_url',conn,if_exists='append')


86

In [24]:
df['first_alphabet'] = df['title'].str[0]
pd.pivot_table(df,index=['first_alphabet'],values=['status'],aggfunc='count')

,status
first_alphabet,
A,325
B,311
C,367
D,133
E,88
F,236
G,334
H,584
I,620


In [13]:
df[df['first_alphabet']=='“']

,index,url,title,status,first_alphabet
780,780,http://cyberhymnal.org/htm/c/o/m/come.htm,“COME”,0.0,“
1359,1359,http://cyberhymnal.org/htm/f/m/c/fmcstndr.htm,“FOLLOW ME!” A CALL SO TENDER,0.0,“
1372,1372,http://cyberhymnal.org/htm/g/l/o/glorygof.htm,"“For God So Loved!” Oh, Wondrous Theme!",0.0,“
2826,2826,http://cyberhymnal.org/htm/a/c/c/accepted.htm,“In the Beloved” Accepted Am I,0.0,“
7457,7457,http://cyberhymnal.org/htm/w/h/whosoevm.htm,“Whosoever” Meaneth Me,0.0,“


In [11]:
# auth = HTTPProxyAuth("username", "password")
# Consider removing duplicates in url

user_agent_list = ['Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1623.0 Safari/537.36',
                   'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 7.0; InfoPath.3; .NET CLR 3.1.40767; Trident/6.0; en-IN)',
                   'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36',
                   'Mozilla/5.0 (X11; Ubuntu; Linux armv7l; rv:17.0) Gecko/20100101 Firefox/17.0'
                  ]
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

batch = 1000
mini_batch = 20 
lower_limit = 3
upper_limit = 5

iterations = 0
count_query = pd.read_sql_query("select count(*) as cnt from lyrics_abc where status = 0;",conn)
count = int(count_query['cnt'][0])
initial_count = count - batch

while count > initial_count : 

    count_query = pd.read_sql_query("select count(*) as cnt from lyrics_abc where status = 0;",conn)
    count = int(count_query['cnt'][0])

    # Randomize 
    set_new_ip()
    rand_user_agent = randint(0,len(user_agent_list)-1)
    headers['User-Agent'] = user_agent_list[rand_user_agent]
    elapsed_time = 0 
    # Scrap
    scrap = pd.read_sql_query("select key,url from lyrics_abc where status = 0 limit "+str(mini_batch)+";",conn)
    t0 = time.time()
    for key,row in scrap.iterrows():

        pause = randint(lower_limit,upper_limit)
        time.sleep(pause)

        key = row['key']
        url = row['url']
        response = session.get(url, headers=headers)
        elapsed_time += response.elapsed.total_seconds()
        data = response.text
        soup = BeautifulSoup(data,"lxml")
        try : 
            title = soup.title.get_text()
            meta = str(soup.find('meta',attrs={'name':'description'}))
            fullhtml = str(soup)
            song_lyrics =  str(soup.find('div',attrs={"class":"lyrics"}))
            
            cursor.execute("""UPDATE lyrics_abc SET song_name = ? ,meta = ?,lyrics = ?,fullhtml = ? , status = ? WHERE key = ? """,
              (title,meta,song_lyrics,fullhtml,1,key))
            conn.commit()
        except Exception as ex: 
            print(url," ",type(ex).__name__," ",ex.args)
            
    t1 = time.time()
    print("Interations :",iterations," Per Request:" ,elapsed_time/mini_batch," Total:",t1-t0)
    # print("Initial_count ",initial_count," Current:",count)
    iterations = iterations + 1
    

cursor.close()
conn.close()

Interations : 0  Per Request: 0.68766675  Total: 95.71710562705994
Interations : 1  Per Request: 0.5634615000000001  Total: 88.71667408943176
Interations : 2  Per Request: 0.6807460999999999  Total: 89.3471360206604
Interations : 3  Per Request: 0.55332405  Total: 92.68269371986389
Interations : 4  Per Request: 0.6233717  Total: 93.74278616905212
Interations : 5  Per Request: 0.7862366  Total: 99.73147296905518
Interations : 6  Per Request: 1.07692625  Total: 105.85263705253601
Interations : 7  Per Request: 0.7404698  Total: 99.70292115211487
Interations : 8  Per Request: 0.65167915  Total: 97.84562182426453
Interations : 9  Per Request: 0.8424932  Total: 97.16509485244751
Interations : 10  Per Request: 0.7600020999999998  Total: 96.43898367881775
Interations : 11  Per Request: 0.8254916  Total: 95.80634474754333
Interations : 12  Per Request: 0.7051366  Total: 94.8631317615509
Interations : 13  Per Request: 0.7114553500000002  Total: 96.98612689971924
Interations : 14  Per Request: 0.

http://cyberhymnal.org/htm/l/u/h/luhchasa.htm   AttributeError   ("'NoneType' object has no attribute 'get_text'",)
http://cyberhymnal.org/htm/l/u/f/lufafeet.htm   AttributeError   ("'NoneType' object has no attribute 'get_text'",)
Interations : 41  Per Request: 0.5757317499999999  Total: 96.7964415550232
http://cyberhymnal.org/htm/l/t/ltchstri.htm   AttributeError   ("'NoneType' object has no attribute 'get_text'",)
http://cyberhymnal.org/htm/l/t/lthyglor.htm   AttributeError   ("'NoneType' object has no attribute 'get_text'",)
http://cyberhymnal.org/htm/l/t/ltmernen.htm   AttributeError   ("'NoneType' object has no attribute 'get_text'",)
http://cyberhymnal.org/htm/l/t/lthyword.htm   AttributeError   ("'NoneType' object has no attribute 'get_text'",)
http://cyberhymnal.org/htm/l/t/w/ltw2mrem.htm   AttributeError   ("'NoneType' object has no attribute 'get_text'",)
http://cyberhymnal.org/htm/l/t/ltmtwmak.htm   AttributeError   ("'NoneType' object has no attribute 'get_text'",)
http://

http://cyberhymnal.org/htm/m/o/r/morningl.htm   AttributeError   ("'NoneType' object has no attribute 'get_text'",)
http://cyberhymnal.org/htm/m/o/mornlite.htm   AttributeError   ("'NoneType' object has no attribute 'get_text'",)
http://cyberhymnal.org/htm/m/o/mornpray.htm   AttributeError   ("'NoneType' object has no attribute 'get_text'",)
http://cyberhymnal.org/htm/m/o/mornpurp.htm   AttributeError   ("'NoneType' object has no attribute 'get_text'",)
http://cyberhymnal.org/htm/m/o/mornired.htm   AttributeError   ("'NoneType' object has no attribute 'get_text'",)
http://cyberhymnal.org/htm/m/o/r/mornwalk.htm   AttributeError   ("'NoneType' object has no attribute 'get_text'",)
http://cyberhymnal.org/htm/m/r/mrhhdsky.htm   AttributeError   ("'NoneType' object has no attribute 'get_text'",)
http://cyberhymnal.org/htm/m/o/mortawaj.htm   AttributeError   ("'NoneType' object has no attribute 'get_text'",)
http://cyberhymnal.org/htm/m/g/mglolife.htm   AttributeError   ("'NoneType' object h

KeyboardInterrupt: 

In [12]:
conn = sqlite3.connect('lyrics.db')
count_query = pd.read_sql_query("select count(*) as cnt from lyrics_abc where status = 0;",conn)
count_query

,cnt
0,3674


In [37]:
results = pd.read_sql_query("select * from lyrics_abc where status = 1;",conn)
results

,index,url,title,status,song_name,meta,lyrics,fullhtml,key
0,0,http://cyberhymnal.org/htm/a/b/abbafath.htm,"ABBA, FATHER! WE APPROACH THEE",1.0,"Abba, Father! We Approach Thee","<meta content=""Words: James Deck, 1841. Music:...","<div class=""lyrics"">\n<p>“Abba, Father!” We ap...","<?xml version=""1.0"" encoding=""ISO-8859-2"" ?><!...",0
1,1,http://cyberhymnal.org/htm/a/b/abidinme.htm,"ABIDE IN ME, O LORD",1.0,"Abide in Me, O Lord","<meta content=""Words: Harriet Stowe, 1855. Mus...","<div class=""lyrics"">\n<p>Abide in me, O Lord, ...","<?xml version=""1.0"" encoding=""ISO-8859-2"" ?><!...",1
2,2,http://cyberhymnal.org/htm/a/i/abinthee.htm,ABIDE IN THEE,1.0,Abide in Thee,"<meta content=""Words: Joseph Smith, 1869. Musi...","<div class=""lyrics"">\n<p>Abide in Thee, in tha...","<?xml version=""1.0"" encoding=""ISO-8859-2"" ?><!...",2
3,3,http://cyberhymnal.org/htm/a/b/abidenot.htm,ABIDE NOT IN THE REALM OF DREAMS,1.0,Abide Not in the Realm of Dreams,"<meta content=""Words: William Burleigh, 1871. ...","<div class=""lyrics"">\n<p>Abide not in the real...","<?xml version=""1.0"" encoding=""ISO-8859-2"" ?><!...",3
4,4,http://cyberhymnal.org/htm/a/o/aodjesus.htm,"ABIDE, O DEAREST JESUS",1.0,"Abide, O Dearest Jesus","<meta content=""Words: Joshua Stegmann, 1628. M...","<div class=""lyrics"">\n<p>Abide, O dearest Jesu...","<?xml version=""1.0"" encoding=""ISO-8859-2"" ?><!...",4
5,5,http://cyberhymnal.org/htm/a/b/abidewme.htm,ABIDE WITH ME,1.0,Abide with Me,"<meta content=""Words: Henry Lyte, 1847. Music:...",None,"<?xml version=""1.0"" encoding=""ISO-8859-2"" ?><!...",5
6,6,http://cyberhymnal.org/htm/a/b/abidewmt.htm,"ABIDE WITH ME, ’TIS EVENTIDE",1.0,"Abide with Me, ’Tis Eventide","<meta content=""Words: Lowrie Hofford, 1884. Mu...","<div class=""lyrics"">\n<p>Abide with me, ’tis e...","<?xml version=""1.0"" encoding=""ISO-8859-2"" ?><!...",6
7,7,http://cyberhymnal.org/htm/a/w/awuosavr.htm,"ABIDE WITH US, OUR SAVIOR",1.0,"Abide with Us, Our Savior","<meta content=""Words: Joshua Stegmann, 1632. M...","<div class=""lyrics"">\n<p>Abide with us, our Sa...","<?xml version=""1.0"" encoding=""ISO-8859-2"" ?><!...",7
8,8,http://cyberhymnal.org/htm/a/b/abidewus.htm,"ABIDE WITH US, THE DAY IS WANING",1.0,"Abide with Us, the Day Is Waning","<meta content=""Words: Caspar Boye, 1834. Music...","<div class=""lyrics"">\n<p>“Abide with us, the d...","<?xml version=""1.0"" encoding=""ISO-8859-2"" ?><!...",8
9,9,http://cyberhymnal.org/htm/a/b/able2del.htm,ABLE TO DELIVER,1.0,Able to Deliver,"<meta content=""Words: Fanny Crosby, 1899. Musi...","<div class=""lyrics"">\n<p>O troubled heart, be ...","<?xml version=""1.0"" encoding=""ISO-8859-2"" ?><!...",9


In [34]:
song_composers = soup.body.table.find("td").findAll("p")

sc = []
for s in song_composers:
    sc.append(s.get_text().replace("&shy;",""))
    print(s.get_text().replace("&shy;",""))
# song_music =  soup.find('span',attrs={"class":"music"})
# print(song_music)

song_lyrics =  soup.find('div',attrs={"class":"lyrics"})
print(song_lyrics)

Words: bio("Harriet B. Stowe","s/t/o/stowe_hb"), in the Ply­mouth Col­lect­ion, 1855.
Music: Cuba, in Temp­li Car­mi­na, by bio("George Kingsley","k/i/n/kingsley_g") (North­amp­ton, Mass­a­chu­setts: 1853)lmn("c/u/Cuba").
<div class="lyrics">
<p>Abide in me, O Lord, and I in Thee,<br/>
From this good hour, oh, leave me nevermore;<br/>
Then shall the discord cease, the wound be healed,<br/>
The lifelong bleeding of the soul be o’er.</p>
<p>Abide in me; o’ershadow by Thy love<br/>
Each half formed purpose and dark thought of sin;<br/>
Quench ere it rise each selfish, low desire,<br/>
And keep my soul as Thine, calm and divine.</p>
<p>As some rare perfume in a vase of clay,<br/>
Pervades it with a fragrance not its own,<br/>
So, when Thou dwellest in a mortal soul,<br/>
All Heaven’s own sweetness seems around it thrown.</p>
<p>Abide in me; there have been moments blest<br/>
When I have heard Thy voice and felt Thy power;<br/>
Then evil lost its grasp; and passion, hushed,<br/>
Owned the d

In [37]:
print(sc[0])
composer = re.findall("bio\(\"(.*)\",",sc[0])
year = re.findall("\d{4}",sc[0])
print("Composer is ",composer[0]," Year ",year[0])
final = re.sub("bio\(\"(.*)\",.*?\)",r"\1",sc[0])
print(final)

print(sc[1])
music = re.sub("bio\(\"(.*)\",.*?\)",r"\1",sc[1])
music = re.sub("lmn\(\".*","",music)
print(music)

Words: bio("Harriet B. Stowe","s/t/o/stowe_hb"), in the Ply­mouth Col­lect­ion, 1855.
Composer is  Harriet B. Stowe  Year  1855
Words: Harriet B. Stowe, in the Ply­mouth Col­lect­ion, 1855.
Music: Cuba, in Temp­li Car­mi­na, by bio("George Kingsley","k/i/n/kingsley_g") (North­amp­ton, Mass­a­chu­setts: 1853)lmn("c/u/Cuba").
Music: Cuba, in Temp­li Car­mi­na, by George Kingsley (North­amp­ton, Mass­a­chu­setts: 1853)


## Reference Scraping 

In [ ]:
url = 'http://www.whoishostingthis.com/tools/user-agent/'
response = requests.get(url, headers=headers,auth=auth)
data = response.text
soup = BeautifulSoup(data,"lxml")
useragent = soup.find(attrs={"class":"info-box user-agent"})
ip = soup.find(attrs={"class":"info-box ip"})
print(useragent.text)
print(headers["User-Agent"])
print(ip.span.text)

In [ ]:
### Example Code for using Sessions
# s = requests.Session()
# s.get(url,headers=headers)
# print(s.cookies)
# r = s.post(url, data=payload,auth=auth,headers=headers,cookies=cookie)
# Get Christian Artists only from e-lyrics

In [ ]:
cookie = {'session_name':'session','cookie_1': 'cookie_session_1'}
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
           'header_2':'header_value_2'
          }


payload = {'input1':'value1',
           'input2': 'value2',
          }

r = requests.post(url, data=payload,auth=auth,headers=headers,cookies=cookie)

data = r.text
soup = BeautifulSoup(data,"lxml")
#Debug
r.request.headers
r.headers

file = open("output.html","w") 
file.write(soup.prettify()) 
file.close() 

In [ ]:
#Create SQLite
conn = sqlite3.connect('lyrics.db')
eumo = pd.read_csv('EU_MO.csv')
eumo.to_sql('test',conn,if_exists='replace')

cur = conn.cursor()
cur.execute('delete from test')
conn.commit

pd.read_sql_query("select * from test;",conn)

## Scrap AZ Lyrics

## Christian Lyrics Sites

In [ ]:
# http://worship-songs-resources.worshiptogether.com/nav/category/songs/0
# http://www.christianlyricsonline.com/a.html
# http://www.christian-gospelmusiclyrics.com/contemporary/0-9-lyrics-christian-contemporary-music.html
# https://www.thoughtco.com/best-free-christian-gospel-lyrics-websites-709507



## Scrap Company Details

In [141]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
from random import randint
#Websites
#https://www.sgpbusiness.com

#Read up <site>/robots.txt

In [142]:
search_url = "http://www.yelu.sg/category/postal-services/"
base_url = "http://www.yelu.sg"

html = urlopen(search_url)
bsObj = BeautifulSoup(html.read(),"lxml")

In [143]:
page = bsObj.findAll("a",{"class":"page_no"})

max = 0 
for i in page:
    page_no = int(i.get_text())
    if (page_no > max):
        max = page_no
print(max)
# page.findAll("div")

9


In [ ]:
f1 = open('scrap.txt','w')
for x in range(1,max+1):
    pg_search = search_url+str(x)
    print(search_url+str(x))
    #Loop through the pages
    pg_url = urlopen(pg_search)
    pgObj = BeautifulSoup(pg_url.read(),"lxml")
    time.sleep(5)
    pages = pgObj.findAll("h4")
    #Loop through the link in each page
    for i in pages:
        ref = i.a.get("href")
        ran_sleep = randint(10,12)
        time.sleep(ran_sleep)
        coy = urlopen(base_url+ref)
        coyObj = BeautifulSoup(coy.read(),"lxml")
        for s in scrapitem:
          try:
            if len(s.find_all('div')) == 0 :
              header = s.find('span').get_text()
              description = s.get_text()[len(header):]
            else: 
              header = s.find_all('div')[0].get_text()  
              if s.find('span') == None :
                description = s.find_all('div')[1].get_text()
              else : 
                description = s.find('span').get_text()    
            #f1.write(base_url+ref)
            f1.write(header,":",description,"\n")
            if header == 'E-mail' :
                break
          except AttributeError as e:
            continue
          except Exception as ea:
            continue
f1.close()

http://www.yelu.sg/category/postal-services/1
http://www.yelu.sg/category/postal-services/2
http://www.yelu.sg/category/postal-services/3
http://www.yelu.sg/category/postal-services/4
http://www.yelu.sg/category/postal-services/5

In [88]:
# t.find_all('div')[0].get_text()
# t.find_all('div')[1].get_text()
x = t.find('span')
if (x == None):
  print("No Span")
else :
  print("Yes")
# t.div.find_next_siblings()

No Span


In [98]:
pg_url = urlopen("http://www.yelu.sg/category/postal-services/9")

In [101]:
pgObj = BeautifulSoup(pg_url.read(),"lxml")
pages = pgObj.findAll("h4")
pages

[]

In [6]:
def getTitle(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        return None
    try:
        bsObj = BeautifulSoup(html.read())
        title = bsObj.body.h1
    except AttributeError as e:
        return None
    return title

In [63]:
time.sleep(3)